Needed imports and downloads:

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import re

In [2]:
#>>> pip install pyrosm 

In [3]:
#>>> pip install osmpythontools

In [4]:
#>>> pip install ipyleaflet

In [5]:
#>>> pip install leafmap

In [9]:
from OSMPythonTools.nominatim import Nominatim
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapely.wkt
import geopandas as gpd
import matplotlib.pyplot as plt
from pyrosm import OSM, get_data
from ipyleaflet import Map, Marker, basemaps, Polyline, basemap_to_tiles, GeoJSON
import leafmap
from pyproj import Proj, transform

ModuleNotFoundError: No module named 'geopandas'

Reading Noise_Reports.csv

In [ ]:
df = pd.read_csv('data/Noise_Reports.csv')

Deleting columns we won't need

In [ ]:
del df['Media URL'] #Almost all are nulls, rest are just photos
del df['Status Notes']
del df['Status']
del df['Updated']
del df['Responsible Agency']
del df['Source']

Deleting points (0.0, 0.0) and null points

In [ ]:
df = df[df.Point != "(0.0, 0.0)"]
df.dropna(subset = ["Point"], inplace=True)

Renaming columns with whitespaces in names

In [ ]:
df.rename(columns = {
    'Request Type' : 'Request_Type', 
    'Request Details':'Request_Details',
    'Supervisor District' : 'Supervisor_District'
    }, inplace = True)

Changing strings to numbers in some columns using OneHotEncoder and LabelEncoder

In [ ]:
opened_encoder = OneHotEncoder()
opened_reshaped = np.array(df['Category']).reshape(-1, 1)
opened_values = opened_encoder.fit_transform(opened_reshaped)

In [ ]:
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])
le2 = LabelEncoder()
df['Request_Type'] = le2.fit_transform(df['Request_Type'])
le3 = LabelEncoder()
df['Request_Details'] = le3.fit_transform(df['Request_Details'])
le4 = LabelEncoder()
df['Neighborhood'] = le4.fit_transform(df['Neighborhood'])
df.head()

Changing column 'Point' (object) to tuple(float,float) and re-indexing columns

In [ ]:
points = []
for i in df.Point:
  points.append((re.findall(r'[+-]?[0-9]+\.[0-9]+', i)))
points = [(float(i), float(j)) for i,j in points] #object to tuple(float,float)
df['Point'] = points
df.reset_index(inplace=True) #re-indexing columns

Checking if poinst are in San Francisco and deleting those, who aren't

In [ ]:
nominatim = Nominatim()
san_fran = nominatim.query("San Francisco, United States", wkt=True)
polygon = shapely.wkt.loads(san_fran.wkt())
for i, point in enumerate(df.Point):
  point = Point(point[1], point[0])
  if not polygon.contains(point):
    df.drop(axis=0, index=i, inplace=True) #deleting points outside San Francisco
df.reset_index(inplace=True) #re-indexing columns

Matching points to buildings - nie odpalać na razie

In [ ]:
#buildings = []
#for i in df.itertuples():
#  print('\n', i.Index, i.Address)
#  building = nominatim.query(i.Address, wkt=True)
#  print(building.displayName())
#  if building.wkt():
#    building_polygon = shapely.wkt.loads(building.wkt())
#    point = Point(i.Point[1], i.Point[0])
#    print(building_polygon, point)
#    if building_polygon.contains(point):
#      print("correct")
#      continue
#    else:
#      print("not in")
#  else:
#    print("incorrect address/no geometry")
#  new_building = nominatim.query(i.Point[0], i.Point[1], reverse=True)
#  if new_building.displayName() == building.displayName():
#    print("correctly assigned")
#    continue
#  buildings.append(new_building.displayName())
#  #print(new_building.displayName())

In [ ]:
Latitude =[]
Longitude = []
for i in range(len(df.Point)):
      Latitude.append(df.Point[i][0])
      Longitude.append(df.Point[i][1])

In [ ]:
df["Latitude"] = Latitude
df["Longitude"] = Longitude

In [ ]:
df

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='epsg:4326')

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

ax = world[world.continent == 'North America'].plot(
    color='white', edgecolor='black')

gdf.plot(ax=ax, color='red')

plt.show()

In [ ]:
fp = get_data("San Francisco", directory='data')
osm = OSM('data/SanFrancisco.osm.pbf')

In [ ]:
gdf_buildings = osm.get_buildings()

In [ ]:
gdf_building = gdf_buildings[['id', 'geometry']]

In [ ]:
#gdf_building.plot(figsize=(16, 12))

In [ ]:
gdf = gdf.to_crs('epsg:7131')

In [ ]:
gdf['buffer_geom'] = gdf.geometry.buffer(1000)

In [ ]:
gdf_building = gdf_building.to_crs('epsg:7131')

In [ ]:
gdf_building

In [ ]:
gdf.head(10)

In [ ]:
df_joins = gpd.sjoin(
    gdf_building[['id','geometry']],
    gdf.set_geometry('geometry'))

In [ ]:
df_joins

In [ ]:
map = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(37.758537, -122.447664),
    zoom=12,
    )

for row in df_joins.head(1000).iterrows():

    marker = Marker()#do zmiany
    marker.location = row['geometry']
    
    map.add_layer(marker)

map.layout.width = '80%'
map.layout.height = '1000px'
map